In [10]:
import pandas as pd
import numpy as np


#### Encoding/Cleaning
- load json data / basic cleaning (sort, drop id, reset indices etc.)
- calculate event duration and interarrival times
- convert and bin values
- concatenate rows into strings (words for NLP) and return as a pd.series

In [18]:
# sort, drop id column and reset index (supercollider seems to occasionally drop events )
df_raw = pd.read_json('./data/grain_improv1.json',orient='index').sort_values('noteOn_timestamp').drop('id',axis=1).reset_index(drop=True)

#calculate duration: noteOn[i] - noteOff[i]
df_raw['duration'] = df_raw.apply(lambda row: row['noteOff_timestamp'] - row['noteOn_timestamp'],axis=1)

# calculate interevent duration (wait time between events): noteOn[i] - noteOn[i-1] ...
# shifting and then dividing by -1 
df_raw['inter_event_duration'] = (df_raw['noteOn_timestamp'] - df_raw['noteOn_timestamp'].shift(-1))/ -1

# timestamps no longer needed
df_raw = df_raw.drop(['noteOn_timestamp','noteOff_timestamp'],axis=1)

#new_index = 
df_raw

,amp,freq,freq_dev,grain_dur,grain_dur_dev,grain_rate,grain_rate_dev,n_voices,rel,duration,inter_event_duration
0,0,55,0,0,0,0,0,0,0,4.182018,4.590028
1,0,68,0,0,0,0,0,0,0,0.550961,0.792000
2,0,60,0,0,0,0,0,0,0,1.258005,1.372955
3,0,49,0,0,0,0,0,0,0,2.619004,4.496038
4,0,72,0,0,0,0,0,0,0,0.483967,0.522970
5,0,66,0,0,0,0,0,0,0,0.608006,0.731994
6,0,60,0,0,0,0,0,0,0,0.569046,6.575028
7,0,63,51,37,48,60,38,15,3,2.632025,3.233976
8,0,66,51,37,48,60,38,15,3,1.144048,1.464007
9,0,68,51,37,48,60,38,15,3,0.559001,1.051028


In [70]:
import string
#used for amplitude
bin4 = [i for i in range(0,127+32,32)]  # need to add one 'step size' 
label4 = [string.ascii_lowercase[i] for i in range(4)]
# used for voices
bin8 = [i for i in range(0,127+16,16)]
label8 = [string.ascii_lowercase[i] for i in range(8)]

# used for grain rate/dur and deviations
bin16 = [i for i in range(0,127+8,8)]
label16 = [string.ascii_lowercase[i] for i in range(16)]

dur_bins = np.logspace(-1,1.25,12)  # need to test edge cases for this one.. 
dur_labels = [string.ascii_lowercase[i] for i in range(11)]

In [71]:
# load the binned df using cut -- keep freq as integer value (for now.. )
binned_df['amp'] = pd.cut(df_raw['amp'], bin4,labels=label4, include_lowest=True)
binned_df['freq_dev'] = pd.cut(df_raw['freq_dev'], bin16,labels=label16,include_lowest=True)
binned_df['grain_dur'] = pd.cut(df_raw['grain_dur'], bin16,labels=label16,include_lowest=True)
binned_df['grain_dur_dev'] = pd.cut(df_raw['grain_dur_dev'],bin16,labels=label16,include_lowest=True)
binned_df['grain_rate']  = pd.cut(df_raw['grain_rate'],bin16,labels=label16,include_lowest=True)
binned_df['grain_rate_dev'] = pd.cut(df_raw['grain_rate_dev'],bin16,labels=label16,include_lowest=True)
binned_df['n_voices'] = pd.cut(df_raw['n_voices'],bin8,labels=label8,include_lowest=True)
binned_df['rel'] = pd.cut(df_raw['rel'], bin4,labels=label4,include_lowest=True)

binned_df['duration'] = pd.cut(df_raw['duration'],dur_bins,labels=dur_labels,include_lowest=True)
binned_df['inter_event_duration'] = pd.cut(df_raw['inter_event_duration'],dur_bins,labels=dur_labels,include_lowest=True)

binned_df['freq'] = df_raw['freq']
pd.cut(df_raw['amp'], bin4,labels=label4)

binned_df

,amp,freq,freq_dev,grain_dur,grain_dur_dev,grain_rate,grain_rate_dev,n_voices,rel,duration,inter_event_duration
0,a,55,a,a,a,a,a,a,a,h,i
1,a,68,a,a,a,a,a,a,a,d,e
2,a,60,a,a,a,a,a,a,a,f,f
3,a,49,a,a,a,a,a,a,a,g,i
4,a,72,a,a,a,a,a,a,a,d,d
5,a,66,a,a,a,a,a,a,a,d,e
6,a,60,a,a,a,a,a,a,a,d,i
7,a,63,g,e,f,h,e,a,a,g,h
8,a,66,g,e,f,h,e,a,a,f,f
9,a,68,g,e,f,h,e,a,a,d,e


In [5]:
cols = list(binned_df)
cols[1], cols[0] = cols[0], cols[1]
binned_df = binned_df.ix[:,cols]  # works for this sample, but may need to reorder if json file inputs differently
binned_df = binned_df.dropna().reset_index(drop=True)

In [6]:

word_series = binned_df.iloc[:,0].astype(str) + '_' # freq first then string of categories 
for i in range(1, len(binned_df.columns)):
    word_series += binned_df.iloc[:,i].astype(str)

In [8]:
# This code persists the bins used for model inputs - translate.binning_specs.py
# The application will encode this bin list for future decoding -- translate.decoder.py 
word_series

import re
bin_list = pd.cut(df_raw['grain_dur'], bin16).cat.categories.tolist()

test = '[0.1, 0.5)'
re.findall('[-+]?\d+[\.]?\d*',test)
# [i for j in xx for i in k]
bin_list = [re.findall('[-+]?\d+[\.]?\d*',i) for i in bin_list]

for i in range(len(bin_list)):
    for j in range(len(bin_list[i])):
        bin_list[i][j] = int(bin_list[i][j])
    bin_list[i] = tuple(bin_list[i])
bin_list

[(0, 8),
 (8, 16),
 (16, 24),
 (24, 32),
 (32, 40),
 (40, 48),
 (48, 56),
 (56, 64),
 (64, 72),
 (72, 80),
 (80, 88),
 (88, 96),
 (96, 104),
 (104, 112),
 (112, 120)]

#### Training a Skip-Gram Tensorflow model and Generating a Sequence
- The word series is passed to an API that trains a Skip-Gram Model
- The final embedded vectors are evaluated and stored as a top level variable
- A string sequence of user specified length is generated using k-nearest neighbors


In [9]:
word_series

0     72_bdddccaagi
1     78_bidddfbbhj
2     69_befidceagh
3     73_befifceadf
4     74_blfifceaeh
5     70_blfifcgbgh
6     79_blfcfcgbfh
7     84_blfcdcgbdd
8     83_blfcdcgbdf
9     75_befcdcgbfg
10    80_befcfcgbbf
11    78_bbccffgbfg
12    73_bbccfegbdf
13    68_bbccfegbef
Name: freq, dtype: object

#### Decoding

- Specs for the binnings need to be stored in a config file/data structure 
- After tensorflow produces results they need to be decoded back to midi-json format
- midi values are randomized within the binned ranges and returned to supercollider
